# Generating strings that don't have any secrets

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/no_secrets_in_generated_text.ipynb).

In this example, we will use Guardrails to generate strings that don't have any secrets.

This is also a good example to show how to use the `script` element of the `RAIL` specification. In this case, we will use the `script` element to create a custom Validator that checks if a string has any secrets.

## Objective

We want to ask help with an API, but make sure that the generated text has no secrets.

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../rail/output.md).

In this RAIL spec, we:

1. Create a `script` element that creates a custom Validator that checks if a string has any secrets. This is a simple example, but you can use this to create more complex Validators. For more information on creating custom Validators, see the [Validators documentation](../validation.md).
2. Create a `output` schema that returns an object with a `api_help` key.

In [2]:
rail_str = """
<rail version="0.1">

<script language='python'>
from dataclasses import dataclass
from guardrails.validators import Validator, EventDetail, register_validator

import re
from typing import Dict, List

OPENAI_KEY_PATTERN = re.compile(r"sk-[a-zA-Z0-9]{24}")


@register_validator(name="no-code-secrets", data_type="string")
class NoCodeSecrets(Validator):

    def validate(self, key, value, schema) -> Dict:
        global OPENAI_KEY_PATTERN

        if re.search(OPENAI_KEY_PATTERN, value) is not None:
            # Corrected value should replace the OpenAI API key with "sk-xxx"
            correct_value = re.sub(OPENAI_KEY_PATTERN, "sk-xxx", value)
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} is an OpenAI API key.",
                correct_value,
            )

        return schema
</script>


<output>
    <string name="api_help" description="Show an example curl command for using openai Completion API" format="no-code-secrets" on-fail-no-code-secrets="fix" />
</output>


<prompt>

How do I use OpenAI's Completion API?

@complete_json_suffix
</prompt>


</rail>
"""

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [3]:
import guardrails as gd

from rich import print

guard = gd.Guard.from_rail_string(rail_str)

We see the prompt that will be sent to the LLM.

In [4]:
print(guard.base_prompt)

How do I use OpenAI's Completion API?


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="api_help" description="Show an example curl command for using openai Completion API" 
format="no-code-secrets: "/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `None`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{{'foo': 'example one'}}`
- `<list name='bar'><string format='upper-case' /></list>` => `{{"bar": ['STRING ONE', 'STRING TWO', etc.]}}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{{'baz': {{'foo': 'Some String', 'index': 1}}}}`

JSON Object:

## Step 3: Wrap the LLM API call with `Guard`

In [5]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create, engine="text-davinci-003", max_tokens=2048, temperature=0
)

/Users/shreyarajpal/anaconda3/envs/tiff-env/lib/python3.9/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  if isinstance(o, (numpy.bool, numpy.bool_)):


The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [6]:
print(validated_response)

{
    'api_help': 'curl -X POST -H \'Content-Type: application/json\' -d \'{"prompt": "The quick brown fox", 
"max_tokens": 10}\' https://api.openai.com/v1/engines/completion/completions'
}

In [7]:
print(guard.state.most_recent_call.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 1 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ How do I use OpenAI's Completion API?                                                                   │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="api_help" description="Show an example curl command for using openai Completion API"  │ │
    │ │ format="no-code-secrets: "/>                                                                            │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `None`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │ JSON Object:                                                                                            │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ {"api_help": "curl -X POST -H 'Content-Type: application/json' -d '{\"prompt\": \"The quick brown       │ │
    │ │ fox\", \"max_tokens\": 10}' https://api.ope